In [3]:
library(tidyverse)
library(corrr)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [4]:
centered_factor <- function(x) {
  f <- as.factor(x)
  n <- nlevels(f)
  
  if (n %% 2 == 1) {
    # Odd number of levels: include 0
    centered <- seq(-(n %/% 2), n %/% 2)
  } else {
    # Even number of levels: exclude 0
    centered <- seq(-n / 2, n / 2)[-((n / 2) + 1)]
  }
  
  centered[as.integer(f)]
}

corrs <- function(df){
    return(df %>% 
               #mutate(across(everything(), centered_factor)) %>% 
               mutate(across(everything(), ~ as.numeric(as.factor(.)))) %>% 
               correlate() %>%
               as_tibble() %>%
               mutate_if(is.numeric, round, 3))
}

# Main task

In [12]:
cb <- read_csv("/Users/Barbu/Documents/Postdoc/Experiments/Pushmi/MEG/Randomization/cb.csv", col_types = cols()) %>%
    filter(trial_type == 'test')

In [13]:
cb %>% group_by(lateral_position, central_shape, movement) %>% count()

lateral_position,central_shape,movement,n
<chr>,<chr>,<chr>,<int>
left,shape1,left,1440
left,shape1,right,1440
left,shape2,left,1440
left,shape2,right,1440
right,shape1,left,1440
right,shape1,right,1440
right,shape2,left,1440
right,shape2,right,1440


In [14]:
cb %>% 
    mutate(alphabet_order = ifelse(shape_pair == paste0(shape1, "-", shape2), 1, 0)) %>%
    filter(shape_pair == "cercle-croix") %>%
    #select(-trial_block, -trial_type, -trial_number, -test_sentence, -ground_truth, -patient_assignment) %>%
    select(shape_pair, agent, agent_shape, assignment_order, central_shape, outcome,
           lateral_position, movement, correct_key, alphabet_order) %>%
    # mutate(across(everything(), ~ as.numeric(as.factor(.)))) %>%
    # mutate(interaction = (agent_shape - 1) + 2 * (central_shape - 1) + 1) %>%
    # group_by(shape_pair, interaction) %>% count()
    corrs

Warning message in stats::cor(x = x, y = y, use = use, method = method):
“the standard deviation is zero”
Correlation computed with
• Method: 'pearson'
• Missing treated using: 'pairwise.complete.obs'


term,shape_pair,agent,agent_shape,assignment_order,central_shape,outcome,lateral_position,movement,correct_key,alphabet_order
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
shape_pair,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
agent,NA,NA,0,0,0,0,0,0,0,0
agent_shape,NA,0,NA,0,0,0,0,0,0,0
assignment_order,NA,0,0,NA,0,0,0,0,0,0
central_shape,NA,0,0,0,NA,0,0,0,0,0
outcome,NA,0,0,0,0,NA,0,0,0,0
lateral_position,NA,0,0,0,0,0,NA,0,0,0
movement,NA,0,0,0,0,0,0,NA,0,0
correct_key,NA,0,0,0,0,0,0,0,NA,0


In [11]:
cb %>% 
    filter(shape_pair == 'cercle-croix') %>% 
    group_by(central_shape, outcome) %>% count() 

central_shape,outcome,n
<chr>,<chr>,<int>
shape1,pousse,16
shape1,tire,16
shape2,pousse,16
shape2,tire,16


In [44]:
cb %>% 
    #filter(shape_pair == 'cercle-croix') %>%
    select(-block_number, -trial_type, -trial_number, -test_sentence, -ground_truth, -patient_shape) %>%
    select(shape_pair, agent, agent_shape, assignment_order, central_shape, outcome,
           lateral_position, movement, correct_key) %>%
#     mutate(across(everything(), ~ as.numeric(as.factor(.)))) %>%
#     mutate(interaction = (outcome - 1) + 2 * (agent_assignment - 1) + 1) %>%
#     group_by(shape_pair, interaction) %>% count()
    corrs

Correlation computed with
• Method: 'pearson'
• Missing treated using: 'pairwise.complete.obs'


term,shape_pair,agent,agent_shape,assignment_order,central_shape,outcome,lateral_position,movement,correct_key
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
shape_pair,NA,0,0,0,0,0,0,0,0
agent,0,NA,0,0,0,0,0,0,0
agent_shape,0,0,NA,0,0,0,0,0,0
assignment_order,0,0,0,NA,0,0,0,0,0
central_shape,0,0,0,0,NA,0,0,0,0
outcome,0,0,0,0,0,NA,0,0,0
lateral_position,0,0,0,0,0,0,NA,0,0
movement,0,0,0,0,0,0,0,NA,0
correct_key,0,0,0,0,0,0,0,0,NA


In [20]:
for(i in 1:30) {
    v <- cb %>% 
        filter(ID == i) %>%
        select(-trial_block, -trial_type, -trial_number, -test_sentence, -ground_truth, -patient_assignment) %>%
        select(shape_pair, agent, agent_assignment, assignment_order, central_shape, outcome,
            peripheral_position, movement, correct_key) %>%
    #     mutate(across(everything(), ~ as.numeric(as.factor(.)))) %>%
    #     mutate(interaction = (outcome - 1) + 2 * (agent_assignment - 1) + 1) %>%
    #     group_by(shape_pair, interaction) %>% count()
        corrs %>%
        pivot_longer(-term) %>%
        filter(!is.na(value)) %>%
        pull(value) %>% max()
    print(v)}

Correlation computed with
* Method: 'pearson'
* Missing treated using: 'pairwise.complete.obs'


[1] 0


Correlation computed with
* Method: 'pearson'
* Missing treated using: 'pairwise.complete.obs'


[1] 0


Correlation computed with
* Method: 'pearson'
* Missing treated using: 'pairwise.complete.obs'


...

[1] 0.125


Correlation computed with
* Method: 'pearson'
* Missing treated using: 'pairwise.complete.obs'


[1] 0.208


Correlation computed with
* Method: 'pearson'
* Missing treated using: 'pairwise.complete.obs'


[1] 0.083


In [ ]:
# for(i in 1:10000){
#     selection <- sample(1:130, 30, replace = FALSE, prob = NULL)

#     bla <- cb %>% 
#         filter(ID %in% selection) %>%
#         select(central_shape, outcome, peripheral_position) %>%
#         corrs %>% 
#         pivot_longer(-term) %>%
#         filter(!is.na(value)) %>%
#         filter(value != 0)

#     if(nrow(bla) == 0){print(selection)}
# }

Correlation computed with
* Method: 'pearson'
* Missing treated using: 'pairwise.complete.obs'
Correlation computed with
* Method: 'pearson'
* Missing treated using: 'pairwise.complete.obs'
Correlation computed with
* Method: 'pearson'
* Missing treated using: 'pairwise.complete.obs'
Correlation computed with
* Method: 'pearson'
* Missing treated using: 'pairwise.complete.obs'
Correlation computed with
* Method: 'pearson'
* Missing treated using: 'pairwise.complete.obs'
Correlation computed with
* Method: 'pearson'
* Missing treated using: 'pairwise.complete.obs'
Correlation computed with
* Method: 'pearson'
* Missing treated using: 'pairwise.complete.obs'
Correlation computed with
* Method: 'pearson'
* Missing treated using: 'pairwise.complete.obs'
Correlation computed with
* Method: 'pearson'
* Missing treated using: 'pairwise.complete.obs'
Correlation computed with
* Method: 'pearson'
* Missing treated using: 'pairwise.complete.obs'
Correlation computed with
* Method: 'pearson'
* Mi

 [1]   6  22  75  88 105   7 101  24 107 118  12  57 121 112 109 122  76 104  52
[20]  67  16  68  11 123  44  61  23 127  70  31


Correlation computed with
* Method: 'pearson'
* Missing treated using: 'pairwise.complete.obs'
Correlation computed with
* Method: 'pearson'
* Missing treated using: 'pairwise.complete.obs'
Correlation computed with
* Method: 'pearson'
* Missing treated using: 'pairwise.complete.obs'
Correlation computed with
* Method: 'pearson'
* Missing treated using: 'pairwise.complete.obs'
Correlation computed with
* Method: 'pearson'
* Missing treated using: 'pairwise.complete.obs'
Correlation computed with
* Method: 'pearson'
* Missing treated using: 'pairwise.complete.obs'
Correlation computed with
* Method: 'pearson'
* Missing treated using: 'pairwise.complete.obs'
Correlation computed with
* Method: 'pearson'
* Missing treated using: 'pairwise.complete.obs'
Correlation computed with
* Method: 'pearson'
* Missing treated using: 'pairwise.complete.obs'
Correlation computed with
* Method: 'pearson'
* Missing treated using: 'pairwise.complete.obs'
Correlation computed with
* Method: 'pearson'
* Mi

 [1] 104  32  67  53  28  80  99  65  45  34 124 112  88  19  82  74 115  31  96
[20]  16 109 127  56   7  15  59 119  62 103  49


Correlation computed with
* Method: 'pearson'
* Missing treated using: 'pairwise.complete.obs'
Correlation computed with
* Method: 'pearson'
* Missing treated using: 'pairwise.complete.obs'
Correlation computed with
* Method: 'pearson'
* Missing treated using: 'pairwise.complete.obs'
Correlation computed with
* Method: 'pearson'
* Missing treated using: 'pairwise.complete.obs'
Correlation computed with
* Method: 'pearson'
* Missing treated using: 'pairwise.complete.obs'
Correlation computed with
* Method: 'pearson'
* Missing treated using: 'pairwise.complete.obs'
Correlation computed with
* Method: 'pearson'
* Missing treated using: 'pairwise.complete.obs'
Correlation computed with
* Method: 'pearson'
* Missing treated using: 'pairwise.complete.obs'
Correlation computed with
* Method: 'pearson'
* Missing treated using: 'pairwise.complete.obs'
Correlation computed with
* Method: 'pearson'
* Missing treated using: 'pairwise.complete.obs'
Correlation computed with
* Method: 'pearson'
* Mi

...

Correlation computed with
* Method: 'pearson'
* Missing treated using: 'pairwise.complete.obs'
Correlation computed with
* Method: 'pearson'
* Missing treated using: 'pairwise.complete.obs'
Correlation computed with
* Method: 'pearson'
* Missing treated using: 'pairwise.complete.obs'
Correlation computed with
* Method: 'pearson'
* Missing treated using: 'pairwise.complete.obs'
Correlation computed with
* Method: 'pearson'
* Missing treated using: 'pairwise.complete.obs'
Correlation computed with
* Method: 'pearson'
* Missing treated using: 'pairwise.complete.obs'
Correlation computed with
* Method: 'pearson'
* Missing treated using: 'pairwise.complete.obs'
Correlation computed with
* Method: 'pearson'
* Missing treated using: 'pairwise.complete.obs'
Correlation computed with
* Method: 'pearson'
* Missing treated using: 'pairwise.complete.obs'
Correlation computed with
* Method: 'pearson'
* Missing treated using: 'pairwise.complete.obs'
Correlation computed with
* Method: 'pearson'
* Mi

 [1]  53  38  21  99  46 125  37  32  44 120 100  19  34  59  36  58  86  47  28
[20]  77   8 114  93  87  78  69  74   5  11  76


Correlation computed with
* Method: 'pearson'
* Missing treated using: 'pairwise.complete.obs'
Correlation computed with
* Method: 'pearson'
* Missing treated using: 'pairwise.complete.obs'
Correlation computed with
* Method: 'pearson'
* Missing treated using: 'pairwise.complete.obs'
Correlation computed with
* Method: 'pearson'
* Missing treated using: 'pairwise.complete.obs'
Correlation computed with
* Method: 'pearson'
* Missing treated using: 'pairwise.complete.obs'
Correlation computed with
* Method: 'pearson'
* Missing treated using: 'pairwise.complete.obs'
Correlation computed with
* Method: 'pearson'
* Missing treated using: 'pairwise.complete.obs'
Correlation computed with
* Method: 'pearson'
* Missing treated using: 'pairwise.complete.obs'
Correlation computed with
* Method: 'pearson'
* Missing treated using: 'pairwise.complete.obs'
Correlation computed with
* Method: 'pearson'
* Missing treated using: 'pairwise.complete.obs'
Correlation computed with
* Method: 'pearson'
* Mi

 [1]  14 115  99  97  74  71 102  28 125 100  39   7  19  30  58 128   4 126   9
[20]  84  16  92 110  51  78  82  49  10  66  98


Correlation computed with
* Method: 'pearson'
* Missing treated using: 'pairwise.complete.obs'
Correlation computed with
* Method: 'pearson'
* Missing treated using: 'pairwise.complete.obs'
Correlation computed with
* Method: 'pearson'
* Missing treated using: 'pairwise.complete.obs'
Correlation computed with
* Method: 'pearson'
* Missing treated using: 'pairwise.complete.obs'
Correlation computed with
* Method: 'pearson'
* Missing treated using: 'pairwise.complete.obs'
Correlation computed with
* Method: 'pearson'
* Missing treated using: 'pairwise.complete.obs'
Correlation computed with
* Method: 'pearson'
* Missing treated using: 'pairwise.complete.obs'
Correlation computed with
* Method: 'pearson'
* Missing treated using: 'pairwise.complete.obs'
Correlation computed with
* Method: 'pearson'
* Missing treated using: 'pairwise.complete.obs'
Correlation computed with
* Method: 'pearson'
* Missing treated using: 'pairwise.complete.obs'
Correlation computed with
* Method: 'pearson'
* Mi

In [ ]:
prev_trial <- cb %>%
  mutate(
    same_shapes_forward = shape1 == lag(shape1) & shape2 == lag(shape2),
    same_shapes_reverse = shape1 == lag(shape2) & shape2 == lag(shape1),
    same_labels_forward = label_shape1 == lag(label_shape1) & label_shape2 == lag(label_shape2),
    same_labels_reverse = label_shape1 == lag(label_shape2) & label_shape2 == lag(label_shape1),
    diff_shapes = shape1 != lag(shape1) & shape2 != lag(shape2) &
                  shape1 != lag(shape2) & shape2 != lag(shape1),
    diff_labels = label_shape1 != lag(label_shape1) & label_shape2 != lag(label_shape2) &
                  label_shape1 != lag(label_shape2) & label_shape2 != lag(label_shape1),

    prev = case_when(
      (same_shapes_forward & same_labels_forward) |
      (same_shapes_reverse & same_labels_reverse) ~ "no_change",
      (same_shapes_reverse & same_labels_forward) |
      (same_shapes_forward & same_labels_reverse) ~ "swap",
      (same_shapes_forward | same_shapes_reverse) & diff_labels ~ "same_shapes",
      diff_shapes & (same_labels_forward | same_labels_reverse) ~ "same_referents",
      diff_shapes & diff_labels ~ "different",
      
      TRUE ~ NA
    )
  )

prev_trial %>% group_by(prev) %>% filter(!is.na(prev), prev != "no_change") %>% count()

prev,n
<chr>,<int>
different,7
same_referents,8
same_shapes,4
swap,3


In [ ]:
prev_trial_mild <- cb %>%
  mutate(
    shape1_same = shape1 == lag(shape1) | shape1 == lag(shape2),
    shape2_same = shape2 == lag(shape1) | shape2 == lag(shape2),
    label1_same = label_shape1 == lag(label_shape1) | label_shape1 == lag(label_shape2),
    label2_same = label_shape2 == lag(label_shape1) | label_shape2 == lag(label_shape2),
    shapes_same = shape1_same & shape2_same,
    labels_same = label1_same & label2_same
    )

prev_trial_mild %>% group_by(shape1_same | shape2_same) %>% count()

shape1_same | shape2_same,n
<lgl>,<int>
FALSE,23
TRUE,168
NA,1


# Localizer

In [ ]:
cbloc <- read_csv("/Users/Barbu/Desktop/counterbalancing_localizer.csv", col_types = cols())

In [ ]:
cbloc %>%
    filter(correct == 1) %>%
    mutate(diff = trial_number - lag(trial_number)) %>%
    #summarize(mean(diff, na.rm = TRUE)) %>%
    group_by(diff) %>% count()

diff,n
<dbl>,<int>
1,25
2,27
3,37
4,22
5,8
6,16
7,10
8,10
9,8


# Localizer

In [6]:
cb <- read_csv("/Users/Barbu/Documents/Postdoc/Experiments/Pushmi/MEG/Randomization/counterbalancing.csv", col_types = cols()) %>%
    filter(trial_type == 'localizer')

Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"


In [9]:
cb %>% filter(trial_block > 0) %>%
group_by(shape1) %>% count()

shape1,n
<chr>,<int>
biche,63
carre<U+0301>,61
cercle,62
croix,58
dinde,58
lampe,59
losange,59
pince,60
NA,480
